In [65]:
n = 3

graph = graphs.CompleteBipartiteGraph(2, n)
graph.add_edge(0, 1)
edgelist = graph.edges(labels=False)
matroid = Matroid(graph=edgelist, groundset=edgelist)
print(f"The matroid has {len(edgelist)} groundset elements.")
g = SymmetricGroup(range(n+2))
transpositions = set([(0,1)] + [(i, i+1) for i in range(2, n+1)])
group = g.subgroup(transpositions)
num_conjugacy_classes = len(group.conjugacy_classes())

def action_on_groundset(g, x):
    return tuple(sorted(g(y) for y in x))

def action_on_flats(g, m):
    return tuple(sorted([action_on_groundset(g,x) for x in m]))

def stab(G, m, action):
    return G.subgroup(set(g for g in G if action(g, m) == tuple(m)))

def orbits(G, X, action):
    def orbit(G, m):
        return set(sorted(action(g, m) for g in G))
    return sorted(set(tuple(orbit(G, x)) for x in X))

def perm_representation(G, X, action):
    fn = ClassFunction(G, [0] * num_conjugacy_classes)
    for orbit in orbits(G, X, action):
        stabilizer_group = stab(G, orbit[0], action)
        chi = stabilizer_group.trivial_character()
        fn += chi.induct(G)

    return fn

The matroid has 7 groundset elements.


In [66]:
rank = matroid.rank()

flats = [list(matroid.flats(i)) for i in range(rank + 1)]

def contraction(matroid, flat):
    return matroid.contract(flat)#.simplify()

def restriction(matroid, flat):
    return matroid.delete(matroid.groundset() - flat).simplify()

In [67]:
stab_subgroups = []

for x in orbits(group, flats[2], action_on_flats):
    stabilizer_group = stab(group, x[0], action_on_flats)
    stab_subgroups.append(stabilizer_group)
    print(f"orbit size: {len(x)}")
    print(x[0], stabilizer_group, "\n", stabilizer_group.order(), stabilizer_group.structure_description())
    matroid_cont = contraction(matroid, x[0])
    print("contraction:", matroid_cont, "\n")

pr = perm_representation(group, flats[2], action_on_flats).decompose()
for x, y in pr:
    print(x, y.values())

orbit size: 3
((0, 1), (0, 2), (1, 2)) Subgroup generated by [(), (3,4), (0,1), (0,1)(3,4)] of (Subgroup generated by [(3,4), (2,3), (0,1)] of (Symmetric group of order 5! as a permutation group)) 
 4 C2 x C2
contraction: Graphic matroid of rank 2 on 4 elements 

orbit size: 6
((0, 2), (1, 3)) Subgroup generated by [(), (0,1)(2,3)] of (Subgroup generated by [(3,4), (2,3), (0,1)] of (Symmetric group of order 5! as a permutation group)) 
 2 C2
contraction: Graphic matroid of rank 2 on 5 elements 

orbit size: 6
((1, 2), (1, 3)) Subgroup generated by [(), (2,3)] of (Subgroup generated by [(3,4), (2,3), (0,1)] of (Symmetric group of order 5! as a permutation group)) 
 2 C2
contraction: Graphic matroid of rank 2 on 5 elements 

1 [1, -1, 1, -1, 1, -1]
1 [1, 1, 1, -1, -1, -1]
3 [1, 1, 1, 1, 1, 1]
2 [2, 0, -1, -2, 0, 1]
3 [2, 0, -1, 2, 0, -1]


In [68]:
def induced_representation(grp, subgrps):
    fn = ClassFunction(grp, [0] * len(grp.conjugacy_classes()))
    for subgrp in subgrps:
        chi = subgrp.trivial_character()
        fn += chi.induct(grp)

    return fn

def print_representations(fn):
    for x, y in fn.decompose():
        print(f"{x} copies of {y.values()}")

print(f"Subgroups of order {[stab_subgroup.order() for stab_subgroup in stab_subgroups]}")
print_representations(induced_representation(group, stab_subgroups))

Subgroups of order [4, 2, 2]
1 copies of [1, -1, 1, -1, 1, -1]
1 copies of [1, 1, 1, -1, -1, -1]
3 copies of [1, 1, 1, 1, 1, 1]
2 copies of [2, 0, -1, -2, 0, 1]
3 copies of [2, 0, -1, 2, 0, -1]
